<h1 style="color:green; align:center">Netflix Movies and TV Shows - Data Analysis and Visualization</h1> 
<hr style="width:100%; height:5px; border-width:0; color:gray; background-color:gray;">
<center><img style="height:450px;" src="https://media.npr.org/assets/img/2019/07/04/ap_19165551810863-852cd5b90680e782e02303308afd219172d04aa1-s800-c85.jpg"></center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

|Variable             |Description                                        |
|---------------------|---------------------------------------------------|
|Show_id              |Unique ID for every Movie / Tv Show                |
|Type                 |Identifier - A Movie or TV Show                    |
|Title                |Title of the Movie / Tv Show                       |
|Director             |Director of the Movie                              |
|Cast                 |Actors involved in the movie / show                |
|Country              |Country where the movie / show was produced        |
|Date_added           |Date it was added on Netflix                       |
|Release_year         |Actual Release year of the move / show             |
|Rating               |TV Rating of the movie / show                      |
|Duration             |Total Duration - in minutes or number of seasons   |
|Listed_in            |Genre                                              |
|Description          |The summary description                            |

|Rating Label   |Description                                                                          |
|---------------|-------------------------------------------------------------------------------------|
|TV-MA          |This program is intended to be viewed by mature, adult audiences and may be unsuitable for children under 17|
|TV-14          |This program may be unsuitable for children under 14 years of age                    |
|TV-PG          |This program contains material that parents may find unsuitable for younger children. Parental guidance is recommended|
|R              |May be unsuitable for children under the age of 17 (Under 17 requires accompanying parent or adult guardian)|
|TV-Y           |This program is aimed at a very young audience, including children from ages 2–6     |
|TV-PG          |This program contains material that parents may find unsuitable for younger children. Parental guidance is recommended|
|TV-Y7          |This program is most appropriate for children age 7 and up                           |
|TV-G           |This program is suitable for all ages.                                               |

<h1 style=color:blue align="left"> Load Required Libraries </h1>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, plot, init_notebook_mode, iplot

<h2 style=color:green align="left"> Load and Check Data </h2>

In [ ]:
# Load the data from the CSV file
data = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

In [ ]:
# Display the first 5 rows of data
data.head()

<h1 style=color:blue align="left"> Exploratory Data Analysis </h1>

In [ ]:
# Display the last 5 rows of data
data.tail()

<h2 style=color:green align="left"> Find Missing Values </h2>

In [ ]:
# There are 7787 movies / TV shows in the dataset
print('Total Number of Rows:', data.shape[0])
print('Total Number of Columns:', data.shape[1])
print('-' * 10)
# Check if there is any duplicated records
print('Duplicated Records:', data.duplicated().any())
print('-' * 10)
print('Missing Values:', data.isnull().values.sum())
print('\n')
print('Unique Values: \n\n', data.nunique())
print('-' * 10)
# Most of the missing values is in the director, cast and country column
print('Missing rows in each column: \n')
for i in data.columns:
    # Total number of null values in the column 
    nullNum = data[i].isnull().sum()
    nullRate = nullNum / len(data) * 100 
    if nullRate > 0:
        print(f"{i} {nullNum} ({nullRate:.2f}%)")

In [ ]:
# We can observe 11 of the datatype are object and 1 is integer
data.info()

<h2 style=color:green align="left"> Fill Missing Values </h2>

<h3 style=color:blue align="left"> date_added </h3>

In [ ]:
# Data frame of date_added column
data["date_added"].value_counts().to_frame()

In [ ]:
# Replace null values in data_added column with January 1, mode of {release_year} column
data["date_added"] = data["date_added"].fillna(data["date_added"].mode()[0])

In [ ]:
# Check if all the null values in date_added column are filled
data["date_added"].isnull().sum()

In [ ]:
import re

months = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
dateList = []

# Convert the date_added column from object type to datetime
for i in data['date_added'].values:
    str1 = re.findall('([a-zA-Z]+)\s[0-9]+\,\s[0-9]+', i)
    str2 = re.findall('[a-zA-Z]+\s([0-9]+)\,\s[0-9]+', i)
    str3 = re.findall('[a-zA-Z]+\s[0-9]+\,\s([0-9]+)', i)
    date = '{}-{}-{}'.format(str3[0], months[str1[0]], str2[0])
    dateList.append(date)
    
data['date_added_cleaned'] = dateList
data = data.drop('date_added', axis = 1)
data['date_added_cleaned'] = data['date_added_cleaned'].astype('datetime64[ns]')

data.head(3)

<h3 style=color:blue align="left"> country </h3>

In [ ]:
# Data frame of country column
data["country"].value_counts().to_frame()

In [ ]:
# Replace null values in country column by Japan for Anime
for i, j in zip(data['country'].values, data.index):
    if i == np.nan:
        if ('Anime' in data.loc[j, 'listed_in']) or ('anime' in data.loc[j, 'listed_in']):
            data.loc[j, 'country'] = 'Japan'
        else:
            continue
    else:
        continue

# Replace null values in country column with the word Unknown
data['country'] = data['country'].fillna('Unknown')

In [ ]:
# Check if all the null values in country column are filled
data['country'].isnull().sum()

<h3 style=color:blue align="left"> rating </h3>

In [ ]:
# Data frame of rating column
rating_order = data["rating"].value_counts().to_frame()
rating_order

In [ ]:
# Replace null values in rating column with the mode: TV-MA
data['rating'] = data['rating'].fillna(data['rating'].mode()[0])

In [ ]:
# Check if all the null values in rating column are filled
data['rating'].isnull().sum()

<h3 style=color:blue align="left"> director </h3>

In [ ]:
# Data frame of director column
data["director"].value_counts().to_frame()

In [ ]:
# Replace null values in director column with word Unknown
data['director'] = data['director'].fillna('Unknown')

In [ ]:
# Check if all the null values in director column are filled
data["director"].isnull().sum()

<h3 style=color:blue align="left"> cast </h3>

In [ ]:
# Data frame of cast column
data["cast"].value_counts().to_frame()

In [ ]:
# Replace null values in cast column with word Unknown
data['cast'] = data['cast'].fillna('Unknown')

# Add a column for number of cast
def cast_counter(cast):
    if cast == 'Unknown':
        return 0
    else:
        castList = cast.split(', ')
        castLength = len(castList)
        return castLength

data['number_of_cast'] = data['cast'].apply(cast_counter)

In [ ]:
# Check if all the null values in cast column are filled
data["cast"].isnull().sum()

--------------------------

<h1 style=color:blue align="left"> Visualizations </h1>

<h2 style=color:green align="left"> Comparison of Movies and TV Shows </h2>

In [ ]:
# Plot the bar chart for the number of movies and TV shows
plt.figure(figsize = (8,6))
sns.countplot(x = "type", data = data)

# Get current axis on current figure
ax = plt.gca()

# The maximum scale of y axis
y_max = data['type'].value_counts().max() 
ax.set_ylim([0, y_max + 1000])

# Iterate through the list of axes' patches
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize = 12, color = 'black', ha = 'center', va = 'bottom')

# There are more movies than TV shows in Netflix
plt.title('Comparison of Total TV Shows & Movies', size = '15')
plt.show()

In [ ]:
# Plot the pie chart for the distribution of movies and TV shows
movies_count = data[data.type == 'Movie']
tvshows_count = data[data.type == 'TV Show']

colors = ['red', 'black']
trace = go.Pie(labels = ['Movie', 'TV Show'], 
               values = [movies_count.type.count(), tvshows_count.type.count()],
               hoverinfo = 'label + percent', textinfo = 'label + percent', marker = dict(colors = colors,
                line = dict(color = 'blue', width = 2)))

fig = go.Figure(data = [trace])
iplot(fig)

In [ ]:
# Plot the bar chart for the TV rating distribution 
plt.figure(figsize = (12,8))
sns.countplot(data.rating, order = rating_order.index)
plt.title("Rating for Movies And TV Shows")
plt.xlabel("Rating")
plt.ylabel("Total Count")

# The content for mature adult is the most abundent
plt.show()

In [ ]:
# Plot the bar chart for the TV rating distribution of movies and TV shows separately
plt.figure(figsize = (12,8))
sns.countplot(x = 'rating', data = data, hue = 'type')
plt.xlabel('Rating')
plt.ylabel('Total Count')
plt.show()

<h2 style=color:green align="left"> Movies and TV Shows Content Comparison </h2>

In [ ]:
# List the years with most content released
year_wise_content = data.release_year.value_counts().index[0:20]
year_wise_content

In [ ]:
# Plot the bar chart for the release year distribution of movies and TV shows
plt.figure(figsize = (12,10))

# Most content is released from 2016 to 2020
sns.countplot(data = data, y = 'release_year', order = year_wise_content)

In [ ]:
# Plot the bar chart for the release year distribution of movies and TV shows separately
plt.figure(figsize = (8,6))

import datetime
startYear = datetime.datetime.now().year - 5 + 1
lastFiveYears =  data[data.release_year >= startYear]

sns.catplot(data = lastFiveYears, kind = 'count',
    x = 'release_year', hue = 'type', height = 6)

ax = plt.gca()

# The maximum scale of y axis
ax.set_ylim([0, 1000])

# Iterate through the list of axes' patches
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize = 12, color = 'black', ha = 'center', va = 'bottom')

# The movies produced are decreasing while the TV shows produced are increasing in the last 5 years
plt.title('Last 5 years trends in Netflix', size = '15')
plt.show()

In [ ]:
# Plot the bar and pie chart for the conuntry distribution of movies
from collections import Counter
country_data = data['country']
country_counting = pd.Series(dict(Counter(','.join(country_data).replace(' ,',',').replace(', ',',').split(',')))).sort_values(ascending = False)
top20_country = country_counting[0:20]

from matplotlib import gridspec
fig = plt.figure(figsize = (20, 6))
gs = gridspec.GridSpec(nrows = 1, ncols = 2,
                       height_ratios = [6], 
                       width_ratios = [10, 5])

ax = plt.subplot(gs[0])
sns.barplot(top20_country.index, top20_country, ax = ax)
ax.set_xticklabels(top20_country.index, rotation = '90')
ax.set_title('Top 20 producing countries', fontsize = 15, fontweight='bold')

ax2 = plt.subplot(gs[1])
ax2.pie(top20_country, labels = top20_country.index)
ax2.axis('equal') 

# United States produced the most content
plt.show()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer # Similar to One-Hot Encoding

def relation_heatmap(df, title):
    df['genre'] = df['listed_in'].apply(lambda x :  x.replace(' ,',',').replace(', ',',').split(',')) 
    Types = []
    for i in df['genre']: Types += i
    Types = set(Types)
    print(f"There are {len(Types)} types in the Netflix {title} Dataset")    
    test = df['genre']
    mlb = MultiLabelBinarizer()
    res = pd.DataFrame(mlb.fit_transform(test), columns = mlb.classes_, index = test.index)
    corr = res.corr()
    mask = np.zeros_like(corr, dtype = np.bool)
    mask[np.triu_indices_from(mask)] = True
    fig, ax = plt.subplots(figsize = (15, 14))
    pl = sns.heatmap(corr, mask = mask, cmap = "coolwarm", vmax = .5, vmin = -.5, center = 0,
                    square = True, linewidths = .7, cbar_kws = {"shrink": 0.6})
    
    plt.show()

In [ ]:
# Movie Genre Relatation
# Dramas is not likely to be Documentaries
# Dramas is very likely to be Independent and International Movies
relation_heatmap(movies_count, 'Movie')

In [ ]:
# TV Show Genre Relation
# Kids' TV is not likely to be Internation TV Shows
# Science and Nature TV is very likely to be Docuseries
relation_heatmap(tvshows_count, 'TV Show')

In [ ]:
# Plot the histogram for the durtation of movies
# Most movies last for 75 - 120 mins
sns.set(style = "darkgrid")
sns.kdeplot(data = movies_count['duration'].str.replace(' min','').astype(int), shade = True)

In [ ]:
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

def generateWordCloud(df):
    # Get the unique values of content genres
    genres = list(df['listed_in'])
    gen = []

    for i in genres:
        i = list(i.split(','))
        for j in i:
            gen.append(j.replace(' ',""))
    g = Counter(gen)

    # Generate the Word Cloud for content genres
    text = list(set(gen))
    plt.rcParams['figure.figsize'] = (13, 13)

    wordcloud = WordCloud(background_color = "white").generate(str(text))

    plt.imshow(wordcloud, interpolation = "bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
# Word Cloud for movies genres
generateWordCloud(movies_count)

In [ ]:
# Word Cloud for TV shows genres
generateWordCloud(tvshows_count)

# **Recommedation Engine**

In [ ]:
# import librairies
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as math
import time 
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [14,14]

In [ ]:
# load the data
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
# convert to datetime
df["date_added"] = pd.to_datetime(df['date_added'])
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month
df['day'] = df['date_added'].dt.day
# convert columns "director, listed_in, cast and country" in columns that contain a real list
# the strip function is applied on the elements
# if the value is NaN, the new column contains a empty list []
df['directors'] = df['director'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
df['categories'] = df['listed_in'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
df['actors'] = df['cast'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
df['countries'] = df['country'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])

df.head()

In [ ]:
print(df.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

# Build the tfidf matrix with the descriptions
start_time = time.time()
text_content = df['description']
vector = TfidfVectorizer(max_df=0.4,         # drop words that occur in more than X percent of documents
                             min_df=1,      # only use words that appear at least X times
                             stop_words='english', # remove stop words
                             lowercase=True, # Convert everything to lower case 
                             use_idf=True,   # Use idf
                             norm=u'l2',     # Normalization
                             smooth_idf=True # Prevents divide-by-zero errors
                            )
tfidf = vector.fit_transform(text_content)

# Clustering  Kmeans
k = 200
kmeans = MiniBatchKMeans(n_clusters = k)
kmeans.fit(tfidf)
centers = kmeans.cluster_centers_.argsort()[:,::-1]
terms = vector.get_feature_names()

# print the centers of the clusters
# for i in range(0,k):
#     word_list=[]
#     print("cluster%d:"% i)
#     for j in centers[i,:10]:
#         word_list.append(terms[j])
#     print(word_list) 
    
request_transform = vector.transform(df['description'])
# new column cluster based on the description
df['cluster'] = kmeans.predict(request_transform) 

df['cluster'].value_counts().head()

In [ ]:
# Find similar : get the top_n movies with description similar to the target description 
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [index for index in related_docs_indices][0:top_n]  

In [ ]:
G = nx.Graph(label="MOVIE")
start_time = time.time()
for i, rowi in df.iterrows():
    if (i%1000==0):
        print(" iter {} -- {} seconds --".format(i,time.time() - start_time))
    G.add_node(rowi['title'],key=rowi['show_id'],label="MOVIE",mtype=rowi['type'],rating=rowi['rating'])
#    G.add_node(rowi['cluster'],label="CLUSTER")
#    G.add_edge(rowi['title'], rowi['cluster'], label="DESCRIPTION")
    for element in rowi['actors']:
        G.add_node(element,label="PERSON")
        G.add_edge(rowi['title'], element, label="ACTED_IN")
    for element in rowi['categories']:
        G.add_node(element,label="CAT")
        G.add_edge(rowi['title'], element, label="CAT_IN")
    for element in rowi['directors']:
        G.add_node(element,label="PERSON")
        G.add_edge(rowi['title'], element, label="DIRECTED")
    for element in rowi['countries']:
        G.add_node(element,label="COU")
        G.add_edge(rowi['title'], element, label="COU_IN")
    
    indices = find_similar(tfidf, i, top_n = 5)
    snode="Sim("+rowi['title'][:15].strip()+")"        
    G.add_node(snode,label="SIMILAR")
    G.add_edge(rowi['title'], snode, label="SIMILARITY")
    for element in indices:
        G.add_edge(snode, df['title'].loc[element], label="SIMILARITY")
print(" finish -- {} seconds --".format(time.time() - start_time))   

In [ ]:
def get_all_adj_nodes(list_in):
    sub_graph=set()
    for m in list_in:
        sub_graph.add(m)
        for e in G.neighbors(m):        
                sub_graph.add(e)
    return list(sub_graph)
def draw_sub_graph(sub_graph):
    subgraph = G.subgraph(sub_graph)
    colors=[]
    for e in subgraph.nodes():
        if G.nodes[e]['label']=="MOVIE":
            colors.append('blue')
        elif G.nodes[e]['label']=="PERSON":
            colors.append('red')
        elif G.nodes[e]['label']=="CAT":
            colors.append('green')
        elif G.nodes[e]['label']=="COU":
            colors.append('yellow')
        elif G.nodes[e]['label']=="SIMILAR":
            colors.append('orange')    
        elif G.nodes[e]['label']=="CLUSTER":
            colors.append('orange')

    nx.draw(subgraph, with_labels=True, font_weight='bold',node_color=colors)
    plt.show()

In [ ]:
list_in = ['Inception', 'Star Wars: Episode VIII: The Last Jedi']
sub_graph = get_all_adj_nodes(list_in)
draw_sub_graph(sub_graph)

In [ ]:
def get_recommendation(root):
    commons_dict = {}
    for e in G.neighbors(root):
        for e2 in G.neighbors(e):
            if e2==root:
                continue
            if G.nodes[e2]['label']=="MOVIE":
                commons = commons_dict.get(e2)
                if commons==None:
                    commons_dict.update({e2 : [e]})
                else:
                    commons.append(e)
                    commons_dict.update({e2 : commons})
    movies=[]
    weight=[]
    for key, values in commons_dict.items():
        w=0.0
        for e in values:
            w=w+1/math.log(G.degree(e))
        movies.append(key) 
        weight.append(w)
    
    result = pd.Series(data=np.array(weight),index=movies)
    result.sort_values(inplace=True,ascending=False)        
    return result;

In [ ]:
result = get_recommendation("Inception")
result2 = get_recommendation("Star Wars: Episode VIII: The Last Jedi")
result3 = get_recommendation("Avengers: Infinity War")
result4 = get_recommendation("Spider-Man 3")
print("*"*40+"\n Recommendation for Inception\n"+"*"*40)
print(result.head())
print("*"*40+"\n Recommendation for Star Wars: Episode VIII: The Last Jedi\n"+"*"*40)
print(result2.head())
print("*"*40+"\n Recommendation for Avengers: Infinity War\n"+"*"*40)
print(result3.head())
print("*"*40+"\n Recommendation for Spider-Man 3\n"+"*"*40)
print(result4.head())

In [ ]:
reco = list(result.index[0:2].values)
reco.extend(['Spider-Man 3'])
sub_graph = get_all_adj_nodes(reco)
draw_sub_graph(sub_graph)

In [ ]:
reco = list(result4.index[0:4].values)
reco.extend(['Inception'])
sub_graph = get_all_adj_nodes(reco)
draw_sub_graph(sub_graph)